In [1]:
using Revise

In [2]:
using
    QuantumStates,
    OpticalBlochEquations,
    DifferentialEquations,
    UnitsToValue,
    LinearAlgebra,
    Printf,
    Plots,
    Random,
    StatsBase
;

using Distributions

import MutableNamedTuples: MutableNamedTuple
import StructArrays: StructArray, StructVector
import StaticArrays: @SVector, SVector
import LinearAlgebra: norm, ⋅, adjoint!, diag
import LoopVectorization: @turbo
using BenchmarkTools
using Parameters
using LsqFit

import ProgressMeter: Progress, next!

const λ = 626e-9
const Γ = 2π* 6.4e6
const m = @with_unit 57 "u"
const k = 2π / λ
_μB = (μ_B / h) * 1e-4;

include("CaOH_scan_helper_v2_imbalance.jl")
;

In [3]:
function survived(t_end, times, trajectories)
    _survived = Int64[]
    for i ∈ eachindex(trajectories)
        if abs(times[i][end] - t_end*1e3) <= 1.0
            push!(_survived, i)
        end
    end
    return _survived
end

function cloud_size(trajectories, i)
    std(trajectory[i] for trajectory ∈ trajectories)
end

"""
    Evaluates the density given a set of trajectories.
"""
function density(t_end, results)
    
    times, x_trajectories, y_trajectories, z_trajectories = results.times, results.x_trajectories, results.y_trajectories, results.z_trajectories
    
    # Sample number of photons before the molecule is lost to vibrational dark state
    n_molecules = length(times)
    
    _survived = survived(t_end, times, x_trajectories)
    println(_survived)
    
    surviving_x_trajectories = x_trajectories[_survived]
    surviving_y_trajectories = y_trajectories[_survived]
    surviving_z_trajectories = z_trajectories[_survived]
    
    n = length(_survived)
    _density = Float64(n)
    if n > 1
        idx_end = length(surviving_x_trajectories[1])
        σ_x = cloud_size(surviving_x_trajectories, idx_end)
        σ_y = cloud_size(surviving_y_trajectories, idx_end)
        σ_z = cloud_size(surviving_z_trajectories, idx_end)
        _density /= σ_x * σ_y * σ_z
    end
    return _density
end

function distance(x_trajectory, y_trajectory, z_trajectory, idx)
    return sqrt(x_trajectory[idx]^2 + y_trajectory[idx]^2 + z_trajectory[idx]^2)
end

"""
    Evaluates how many particles are within a 0.3 mm radius.
"""
function goodness(t_end, results)
    
    times, x_trajectories, y_trajectories, z_trajectories = results.times, results.x_trajectories, results.y_trajectories, results.z_trajectories
    
    # Sample number of photons before the molecule is lost to vibrational dark state
    n_molecules = length(times)
    
    _survived = survived(t_end, times, x_trajectories)
    @printf("Survival: %i / %i", length(_survived), n_molecules)
    println()
    
    surviving_x_trajectories = x_trajectories[_survived]
    surviving_y_trajectories = y_trajectories[_survived]
    surviving_z_trajectories = z_trajectories[_survived]
    
    n = 0
    for survived_idx ∈ _survived
        idx_end = length(x_trajectories[survived_idx])
        r = distance(x_trajectories[survived_idx], y_trajectories[survived_idx], z_trajectories[survived_idx], idx_end)
        if r > 1.0
            n += 0.0
        elseif r > 0.5
            n += 0.1 * exp(-results.photons_scattered[survived_idx]/14000)
        elseif r > 0.3
            n += (0.6 - r)* exp(-results.photons_scattered[survived_idx]/14000)
        elseif r <= 0.3
            n += ((0.3 - r) * 3 + 0.3) * exp(-results.photons_scattered[survived_idx]/14000)
        end
    end
    return n/n_molecules
end
;

### Set up the molecule

In [4]:
package = get_CaOH_package()
n_states = length(package.states)
n_excited = package.n_excited
;

In [5]:

"""
    Function to optimize (density).
"""
function f(x)
    s1 = x[1]
    s3 = x[2]

    Δ1 = x[3]*Γ
    Δ3 = x[4]*Γ
    
#     pol1 = sqrt(x[5]) * σ⁺ + sqrt(1-x[5]) * σ⁻
#     pol3 = sqrt(x[6]) * σ⁺ + sqrt(1-x[6]) * σ⁻
    ramp_time = 10e-3
    B_gradient = 30 

    temp = @with_unit 0.050 "mK"
    diameter = @with_unit 0.9 "mm"
    displacement = [0.5, 0.5, 0.5].*1e-3
    kick = [0,0, 0.0, 0.0]
    
    params = MutableNamedTuple(
        t_end=20e-3, n_values=20,
        pol1_x=σ⁺, pol2_x=σ⁺, pol3_x=σ⁺, pol4_x=σ⁻, 
        s1=s1, s2=0.0, s3=s3, s4=0.0,
        Δ1=Δ1, Δ2=0.0, Δ3=Δ3, Δ4=0.0,
        B_gradient=B_gradient,
        temp=temp, 
        diameter=diameter,
        displacement=displacement, 
        kick=kick,
        ramp_time=ramp_time,
        pol_imbalance=0.0, s_imbalance = (0.0, 0.0, 0.0), photon_budget=Inf)
    
    results = simulate_particles_v3(package, params, variable_budget=true)
    
    @printf("s1 = %.2f; s3 = %.2f; Δ1 = %.2fΓ; Δ3 = %.2fΓ",x[1], x[2], x[3], x[4])
    println()
    
    _goodness = goodness(params.t_end, results) * 100
#     if _goodness > 0.2
#         working_dir = "C:\\Users\\halla\\OneDrive\\Desktop\\blueMOT_sim\\Bayesian"
#         test_i = log_test_info(working_dir,1,params)
#         save_results(working_dir, test_i, results)
#     end
# #     println(results.photons_scattered)
    
    @printf("goodness = %.3f", _goodness)
    println()
    
    return _goodness
end
;    

In [6]:
using BayesianOptimization, GaussianProcesses

model = ElasticGPE(4,                            # 6 input dimensions
                   mean = MeanConst(1.),         
                   kernel = SEArd([0.1, 0.1, 0.1, 0.1], 5.),
                   logNoise = 0.,
                   capacity = 3000)              # the initial capacity of the GP is 3000 samples
set_priors!(model.mean, [Normal(1, 1)])

# Optimize the hyperparameters of the GP using maximum a posteriori (MAP) estimates every x steps
modeloptimizer = MAPGPOptimizer(
    every = 1, 
    noisebounds = [-3, 3], # bounds of the logNoise
    kernbounds = [[-3, -3, -3, -3, -5], [3, 3, 3, 3,5]],  # bounds of the parameters GaussianProcesses.get_param_names(model.kernel)
    maxeval = 1000)

opt = BOpt(f,
           model,
           ExpectedImprovement(),                   # type of acquisition
           modeloptimizer,                        
           [0.5, 0.5, 0.5,0.5],        # lowerbounds
           [5.0, 5.0, 3.0,3.0],        # upperbounds         
           repetitions = 1,                          # evaluate the function for each input x times
           maxiterations = 20,                       # evaluate at x input positions
           sense = Max,                              # maximize the function
           acquisitionoptions = (method = :LD_LBFGS, # run optimization of acquisition function with NLopts :LD_LBFGS method
                                 restarts = 5,       # run the NLopt method from x random initial conditions each time
                                 maxtime = 1.0,      # run the NLopt method for at most 1.0 second each time
                                 maxeval = 1000),    # run the NLopt methods for at most 1000 iterations (for other options see https://github.com/JuliaOpt/NLopt.jl)
           verbosity = BayesianOptimization.Progress)

# result = boptimize!(opt);
;

┌ Warning: use values(kwargs) and keys(kwargs) instead of kwargs.data and kwargs.itr
│   caller = #MAPGPOptimizer#19 at gp.jl:37 [inlined]
└ @ Core C:\Users\halla\.julia\packages\BayesianOptimization\nq1HY\src\models\gp.jl:37


In [7]:
# using BayesianOptimization, GaussianProcesses
# using Serialization
# working_dir = "C:\\Users\\halla\\OneDrive\\Desktop\\blueMOT_sim\\Bayesian"
# opt_old = deserialize(joinpath(working_dir, "opt_high_red_det.jl"));
# opt.model = opt_old.model;
# opt.model_optimum = opt_old.model_optimum;
# opt.model_optimizer = opt_old.model_optimizer;
# opt.modeloptimizer = opt_old.modeloptimizer;
# opt.observed_optimum = opt_old.observed_optimum;
# opt.observed_optimizer = opt_old.observed_optimizer;
# opt.acquisition = opt_old.acquisition;
# opt.acquisitionoptions = opt_old.acquisitionoptions;

In [ ]:
using Serialization
working_dir = "C:\\Users\\halla\\OneDrive\\Desktop\\blueMOT_sim\\Bayesian"
# opt = deserialize(joinpath(working_dir, "opt_constrained_red_freq.jl"))
    
while true
    maxiterations!(opt, 30)
    result = boptimize!(opt)
    serialize(joinpath(working_dir, "opt_2freq_4param_flip_pol.jl"), opt)
    println("===== Autosaved =====")
    
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:11


s1 = 0.92; s3 = 2.61; Δ1 = 1.67Γ; Δ3 = 2.14Γ
Survival: 9 / 20
goodness = 0.000


Progress: 100%|█████████████████████████████████████████| Time: 0:01:03


s1 = 3.17; s3 = 4.86; Δ1 = 2.92Γ; Δ3 = 0.89Γ
Survival: 7 / 20
goodness = 0.000


Progress: 100%|█████████████████████████████████████████| Time: 0:01:35


...

[ Info: 2023-11-16T15:24:26.341	iteration: 168	current optimum: 2.3414630154111764
Progress: 100%|█████████████████████████████████████████| Time: 0:01:18


s1 = 3.31; s3 = 1.31; Δ1 = 2.44Γ; Δ3 = 2.63Γ
Survival: 18 / 20
goodness = 0.687


[ Info: 2023-11-16T15:25:45.409	iteration: 169	current optimum: 2.3414630154111764
Progress: 100%|█████████████████████████████████████████| Time: 0:01:14


s1 = 3.49; s3 = 2.38; Δ1 = 2.01Γ; Δ3 = 2.20Γ
Survival: 19 / 20
goodness = 0.877


[ Info: 2023-11-16T15:27:04.857	iteration: 170	current optimum: 2.3414630154111764
Progress: 100%|█████████████████████████████████████████| Time: 0:00:58
[ Info: 2023-11-16T15:28:03.057	iteration: 171	current optimum: 2.3414630154111764


## Scanning configurations

In [ ]:
# fixed:
t_end = 30e-3 #s
n_values = 20
ramp_time = 4e-3 #s
B_gradient = 30 #G/cm

temp = @with_unit 0.5 "mK"
diameter = @with_unit 0.6 "mm"
displacement = [0.0, 0.0, 0.0]
kick = [0,0, 0.0, 0.0]

# variable:
pol1_x_list = [σ⁻]
pol2_x_list = [σ⁺]
pol3_x_list = [σ⁻]
pol4_x_list = [σ⁺]

s1_list = [1.5, 2.0, 2.5]
s2_list = [0.0]
s3_list = [0.05, 0.1]
s4_list = [0.05, 0.1]

Δ1_list = [2.0, 4.0, 6.0] .* Γ
Δ2_list = [0.0]
Δ3_list = [-0.5, -1.0, -1.5] .* Γ
Δ4_list = [0.5, 1.0, 1.5] .* Γ

lists = [pol1_x_list, pol2_x_list, pol3_x_list, pol4_x_list,
        s1_list, s2_list, s3_list, s4_list,
        Δ1_list, Δ2_list, Δ3_list, Δ4_list]

n_scans = 1
for l in lists
   n_scans *= length(l) 
end

comments = "scan d3 +- 1MHz";
saving_dir = make_scan_folder(lists, working_dir, 1, comments);

scan_counter = 0
indicies = [
    1,1,1,1, 
    1,1,1,1, 
    1,1,1,1
];

In [ ]:
while scan_counter < n_scans
    @printf("============= configuration %i / %i =============", scan_counter+1, n_scans)
    println()
    
    pol1_x = pol1_x_list[indicies[1]]
    pol2_x = pol2_x_list[indicies[2]]
    pol3_x = pol3_x_list[indicies[3]]
    pol4_x = pol4_x_list[indicies[4]]
    s1 = s1_list[indicies[5]]
    s2 = s2_list[indicies[6]]
    s3 = s3_list[indicies[7]]
    s4 = s4_list[indicies[8]]
    Δ1 = Δ1_list[indicies[9]]
    Δ2 = Δ2_list[indicies[10]]
    Δ3 = Δ3_list[indicies[11]]
    Δ4 = Δ4_list[indicies[12]]
    
    params = MutableNamedTuple(t_end = t_end, n_values = n_values,
                            pol1_x=pol1_x, pol2_x=pol2_x, pol3_x=pol3_x, pol4_x=pol4_x, 
                            s1=s1, s2=s2, s3=s3, s4=s4,
                            Δ1=Δ1, Δ2=Δ2, Δ3=Δ3, Δ4=Δ4, 
                            B_gradient = B_gradient, 
                            temp=temp, diameter=diameter, 
                            displacement=displacement, kick=kick,
                            ramp_time = ramp_time)

    test_i = log_test_info(saving_dir,1,params)
    
    results = simulate_particles(package, params, params.n_values);

    save_results(saving_dir, test_i, results)
    
    scan_counter += 1 
    next_configuration!(lists, indicies)
    
end
;